In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from numpy import linalg as LA

from scipy.integrate import solve_ivp
from scipy import integrate
from scipy.linalg import null_space

from scipy.optimize import fsolve
from scipy.integrate import solve_bvp

import pickle
import pandas as pd
import cmath

from itertools import chain

In [ ]:
from concurrent.futures import ProcessPoolExecutor

In [ ]:
%matplotlib notebook

In [ ]:
# length
L = 1.7

# orbital number
l = 2

# coupling srength left/right
t_L = 0.7
t_R = t_L

# effective mass
mx = 5

Delta = 1

In [ ]:
phi_0 = np.linspace(0, 2*np.pi, num=101)
phi_0[0] = 0.001
phi_0[-1] -= 0.001
phi_0[50] += 0.001

In [ ]:
lambda_coupl = 0.1 # Andreev-continuum coupling rate
damping_param = 0.01 # 0.01
Omega = 0.1

T_ferm = 0.5
T_bos = 0.2

alpha_0 = 0.001 # 0.0001
omega_c = 1

In [ ]:
def bose_distrib(x, T):
    """ Bose distribution function that handles large x/T to avoid overflow in the exponent """  

    y = x / T
    cutoff = 500.0  # cutoff to avoid overflow in exp
    
    if y >= 0:
        if y > cutoff:
            return 0.0  # for large y, n_B -> 0
        else:
            return 1.0 / np.expm1(y)  # expm1(y) = exp(y) - 1
    else:
        y = -y
        if y > cutoff:
            return -1.0  # for large negative y, expression goes to -1 (since n_B -> 0)
        else:
            return -1.0 - 1.0 / np.expm1(y)

In [ ]:
def fermi_distrib(x, T):
    """ Fermi distribution function that handles large x/T to avoid overflow in the exponent """    
    
    y = x / T
    cutoff = 500.0  # cutoff to avoid overflow in exp
    
    if y > cutoff:
        return 0.0  # for large positive y, 1/(exp(y)+1) -> 0
    elif y < -cutoff:
        return 1.0  # for large negative y, 1/(exp(y)+1) -> 1
    else:
        return 1.0 / (np.exp(y) + 1.0)

In [ ]:
# spectral density function

def spectral_density(x, lambda_coupl, alpha_0, damping_param, Omega, omega_c):
    
    J = lambda_coupl**2 * damping_param * (1/((x - Omega)**2 + damping_param**2/4) - \
                                                   1/((x + Omega)**2 + damping_param**2/4))
    
    J_ohm = alpha_0 * x * np.exp(-np.abs(x)/omega_c)
    
    return J + J_ohm

### Finding free dot eigenenergies and eigenfunctions -  hopping parameters

In [ ]:
def fun(x, y, p):
    k = p[0]
    return np.vstack((y[1], -k**2 * y[0]))

In [ ]:
def bc(ya, yb, p):
#     k = p[0]

# last element - normalization condition (knowing that I will get cos function as a solution)
    return np.array([ya[1], yb[1], yb[0] - np.sqrt(2/L)])

In [ ]:
# Set up the initial mesh and guess for y

x = np.linspace(-L/2, L/2, 10)
y = np.zeros((2, x.size))
y[0, 1] = 1
y[0, 6] = -1

In [ ]:
sol_test = solve_bvp(fun, bc, x, y, p=[np.pi/L])

In [ ]:
# function to find tn at the contacts for the specified l 

def find_tn(l):
    tn = np.zeros((l, 2))
    
    x = np.linspace(-L/2, L/2, 10)
    y = np.zeros((2, x.size))
    y[0, 1] = 1
    y[0, 6] = -1
    
    for i in range(l):
        sol_i = solve_bvp(fun, bc, x, y, p=[(i+1)*np.pi/L])
        
        # tn[l, -L/2]
        tn[i, 0] = sol_i.sol(-L/2)[0]
        
        # tn[l, L/2]
        tn[i, 1] = sol_i.sol(L/2)[0]
        
    return tn

In [ ]:
x_plot = np.linspace(-L/2, L/2, 100)
y_plot = sol_test.sol(x_plot)[0]
plt.plot(x_plot, y_plot)
plt.xlabel("x")
plt.ylabel("y")

# compare with analytical solution:
# sqrt(2/L) * cos(p_n(x-L/2)), p_n = pi*n/L

plt.scatter(x_plot, np.sqrt(2/L)*np.cos(np.pi/L*(x_plot - L/2)), s=10)
plt.grid()

In [ ]:
# hopping parameters

tn = find_tn(l)
tn = -tn
tn

In [ ]:
# Calculate G_L, G_R matrices from tn matrix (2 columns of tn are the hopping parameters (L/R) of each level)

col_L = tn[:, 0]
# pi nu_F?
Gamma_L = t_L**2 * np.outer(col_L, col_L)  # Compute the outer product to form a square matrix
 
col_R = tn[:, 1]
Gamma_R = t_R**2 * np.outer(col_R, col_R) 

In [ ]:
# free dot energies

l_values = np.arange(1, l+1)  
eps_n = (l_values * np.pi / L)**2 / (2 * mx)

### Finding Andreev states

In [ ]:
def G_inv(x, i):
    
    s_L = 1
    s_R = -1
    zero_m = np.zeros_like(Gamma_L)
    
    Lambda = -1 / np.sqrt(Delta**2 - x**2) \
            * (x * np.block([[(Gamma_L + Gamma_R), zero_m], [zero_m, (Gamma_L + Gamma_R)]]) \
            - Delta * np.block([[zero_m, np.exp(-1j * s_L * phi_0[i]/2) * Gamma_L], \
                                [np.exp(1j * s_L * phi_0[i]/2) * Gamma_L, zero_m]]) \
            - Delta * np.block([[zero_m, np.exp(-1j * s_R * phi_0[i]/2) * Gamma_R], \
                                [np.exp(1j * s_R * phi_0[i]/2) * Gamma_R, zero_m]]))
    
    G_inv = x * np.eye(4) - np.block([[np.diag(eps_n), zero_m], [zero_m, -np.diag(eps_n)]]) - Lambda
    
    return LA.det(G_inv)   

In [ ]:
roots = np.zeros((len(phi_0), 2*l))

for i in range(len(phi_0)):
  
    roots[i, 0] = fsolve(G_inv, 0.5, args=(i))  
    roots[i, 1] = fsolve(G_inv, 0.9999, args=(i))  
    
    roots[i, 2] = fsolve(G_inv, -0.5, args=(i))   
    roots[i, 3] = fsolve(G_inv, -0.9999, args=(i))  

In [ ]:
plt.plot(phi_0/np.pi, roots[:,0], linewidth=1.5)
plt.plot(phi_0/np.pi, roots[:,1], linewidth=1.5)


plt.title(r"$t_L={}, t_R={}, m_{{x}} = {}\Delta,$"\
          .format(t_L, t_R, mx)+'\n'+
          r"$\epsilon_1={}\Delta, \epsilon_2 = {}\Delta, L={}\Delta^{{-1}}$"\
          .format(np.round(eps_n[0], 2), np.round(eps_n[1], 2), L), fontsize=13)

plt.grid()
plt.xlabel(r'$\varphi$, $\pi$', fontsize=12)
plt.ylabel(r'$E_A$, $\Delta$', fontsize=12)


### Finding transition rates ABS -> ABS

In [ ]:
# diagonalizing Hamiltonian

def diagonal(x):
    
    eigval_arr = np.zeros((len(phi_0), 2*l))
    eigvec_arr = np.zeros((len(phi_0), 2*l, 2*l), dtype='complex')
    
    s_L = 1
    s_R = -1
    zero_m = np.zeros_like(Gamma_L)
    
    for i in range(len(phi_0)):
    
        Lambda = -1 / np.sqrt(Delta**2 - x[i]**2) \
            * (x[i] * np.block([[(Gamma_L + Gamma_R), zero_m], [zero_m, (Gamma_L + Gamma_R)]]) \
            - Delta * np.block([[zero_m, np.exp(-1j * s_L * phi_0[i]/2) * Gamma_L], \
                                [np.exp(1j * s_L * phi_0[i]/2) * Gamma_L, zero_m]]) \
            - Delta * np.block([[zero_m, np.exp(-1j * s_R * phi_0[i]/2) * Gamma_R], \
                                [np.exp(1j * s_R * phi_0[i]/2) * Gamma_R, zero_m]]))
    
        H = np.block([[np.diag(eps_n), zero_m], [zero_m, -np.diag(eps_n)]]) + Lambda
    
        
        eigval_arr[i], eigvec_arr[i] = LA.eigh(H)
    
    return eigval_arr, eigvec_arr 

In [ ]:
eigv_0, eigvec_0 = diagonal(roots[:, 0])
eigv_1, eigvec_1 = diagonal(roots[:, 1])
eigv_2, eigvec_2 = diagonal(roots[:, 2])
eigv_3, eigvec_3 = diagonal(roots[:, 3])

In [ ]:
plt.plot(phi_0/np.pi, eigv_0[:, 2])
plt.plot(phi_0/np.pi, eigv_1[:, 3])


plt.plot(phi_0/np.pi, eigv_2[:, 1])
plt.plot(phi_0/np.pi, eigv_3[:, 0])
plt.grid()

In [ ]:
# Eigenvectors corresponding to the Andreev energies

eta_0 = eigvec_0[:, :, 2]
eta_1 = eigvec_1[:, :, 3]
eta_2 = eigvec_2[:, :, 1]
eta_3 = eigvec_3[:, :, 0]

eta_arr = np.stack([eta_0, eta_1, eta_2, eta_3], axis=0)

In [ ]:
def curr_matr(x, i):
    
    s_L = 1
    s_R = -1
    zero_m = np.zeros_like(Gamma_L)
    
    Lambda_L = x * np.block([[Gamma_L , zero_m], [zero_m, Gamma_L]]) \
             - Delta * np.block([[zero_m, np.exp(-1j * s_L * phi_0[i]/2) * Gamma_L], \
                                [np.exp(1j * s_L * phi_0[i]/2) * Gamma_L, zero_m]])
    
    Lambda_R = x * np.block([[Gamma_R , zero_m], [zero_m, Gamma_R]]) \
             - Delta * np.block([[zero_m, np.exp(-1j * s_R * phi_0[i]/2) * Gamma_R], \
                                [np.exp(1j * s_R * phi_0[i]/2) * Gamma_R, zero_m]])
    
    I = -1 / np.sqrt(Delta**2 - x**2) / (2*1j) * (s_L * Lambda_L + s_R * Lambda_R)
    
    return I

In [ ]:
def curr_nl(E_n, E_lamb, eta_n, eta_lamb, i):
    
    tau_z = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0], [0, 0, 0, -1]])
    
    matr = tau_z @ curr_matr(E_lamb, i) - curr_matr(E_n, i) @ tau_z
    
    return np.conj(eta_n) @ matr @ eta_lamb    

In [ ]:
I_nl = np.zeros((len(phi_0), 2*l, 2*l), dtype='complex')
G_nl = np.zeros((len(phi_0), 2*l, 2*l))

for i in range(len(phi_0)):
    for n in range(2*l):
        for lamd in range(2*l):
            I_nl[i, n ,lamd] = curr_nl(roots[i, n], roots[i, lamd], eta_arr[n, i], eta_arr[lamd, i], i)

In [ ]:
plt.plot(phi_0/np.pi, np.abs(I_nl[:, 3, 2]))
plt.plot(phi_0/np.pi, np.abs(I_nl[:, 2, 3]))

plt.plot(phi_0/np.pi, np.abs(I_nl[:, 1, 0]))
plt.plot(phi_0/np.pi, np.abs(I_nl[:, 0, 1]))

In [ ]:
for i in range(len(phi_0)):
    for n in range(2*l):
        for lamd in range(2*l):
            
            energy_diff = roots[i, n] - roots[i, lamd]
            J_spect = spectral_density(energy_diff, lambda_coupl, alpha_0, damping_param, Omega, omega_c)
            bose = bose_distrib(energy_diff, T_bos)
            
            G_nl[i, lamd, n] = 2*np.pi * np.abs(I_nl[i, n, lamd])**2 * J_spect * bose 
            
            if math.isnan(G_nl[i, lamd, n]):
                G_nl[i, lamd, n] = 0

In [ ]:
# trans rates 0,1,2,3 -> 0,1,2,3

G_AL_pp = np.zeros((len(phi_0), 4, 4))
G_AL_pp[:, 0:2, 0:2] = G_nl[:, 0:2, 0:2]
G_AL_pp[:, 2:4, 2:4] = G_nl[:, 0:2, 0:2]

# trans rates 0,1,2,3 -> -0,-1,-2,-3

G_AL_pm = np.zeros((len(phi_0), 4, 4))
G_AL_pm[:, 0:2, 2:4] = G_nl[:, 0:2, 2:4]
G_AL_pm[:, 2:4, 0:2] = G_nl[:, 0:2, 2:4]


# trans rates -0,-1,-2,-3 -> 0,1,2,3

G_AL_mp = np.zeros((len(phi_0), 4, 4))
G_AL_mp[:, 2:4, 0:2] = G_nl[:, 2:4, 0:2]
G_AL_mp[:, 0:2, 2:4] = G_nl[:, 2:4, 0:2]

In [ ]:
plt.plot(phi_0/np.pi, G_AL_pp[:, 0, 1], label=r'$\Gamma_{12}$')
plt.plot(phi_0/np.pi, G_AL_pp[:, 1, 0], label=r'$\Gamma_{21}$')

plt.plot(phi_0/np.pi, G_AL_pm[:, 0, 2], label=r'$\Gamma_{1\bar1}$')
plt.plot(phi_0/np.pi, G_AL_pm[:, 0, 3], label=r'$\Gamma_{1\bar2} = \Gamma_{2\bar1}$')
plt.plot(phi_0/np.pi, G_AL_pm[:, 1, 3], label=r'$\Gamma_{2\bar2}$')

plt.plot(phi_0/np.pi, G_AL_mp[:, 0, 2], label=r'$\Gamma_{\bar1 1}$')
plt.plot(phi_0/np.pi, G_AL_mp[:, 0, 3], label=r'$\Gamma_{\bar1 2} = \Gamma_{\bar2 1}$')
plt.plot(phi_0/np.pi, G_AL_mp[:, 1, 3], label=r'$\Gamma_{\bar2 2}$')


plt.legend()

### Finding transition rates ABS -> continuum: DOS

In [ ]:
# DOS matrix

def spectr(x, i):
    
    eigval_arr = np.zeros((4))
    eigvec_arr = np.zeros((4, 4), dtype='complex')
    
    s_L = 1
    s_R = -1
    zero_m = np.zeros_like(Gamma_L)
    
    Lambda = 1 / np.sqrt(x**2 - Delta**2) \
        * (x * np.block([[(Gamma_L + Gamma_R), zero_m], [zero_m, (Gamma_L + Gamma_R)]]) \
        - Delta * np.block([[zero_m, np.exp(-1j * s_L * phi_0[i]/2) * Gamma_L], \
                            [np.exp(1j * s_L * phi_0[i]/2) * Gamma_L, zero_m]]) \
        - Delta * np.block([[zero_m, np.exp(-1j * s_R * phi_0[i]/2) * Gamma_R], \
                            [np.exp(1j * s_R * phi_0[i]/2) * Gamma_R, zero_m]]))

    G_R = LA.inv(x * np.eye(4) - np.block([[np.diag(eps_n), zero_m], [zero_m, -np.diag(eps_n)]]) \
          + 1j * np.sign(x) * Lambda)

    G_A = np.transpose(np.conj(G_R))

    Spectr = 1j * (G_R - G_A)

    eigval_arr, eigvec_arr = LA.eigh(Spectr)
    
    return eigval_arr, eigvec_arr    

In [ ]:
eigv_c = np.zeros((len(phi_0), 4))
eigvect_c = np.zeros((len(phi_0), 4, 4), dtype='complex')

for i in range(len(phi_0)):
    eigv_c[i], eigvect_c[i] = spectr(-1.1, i)

In [ ]:
plt.plot(phi_0/np.pi, eigv_c[:, 0])
plt.plot(phi_0/np.pi, eigv_c[:, 1])
plt.plot(phi_0/np.pi, eigv_c[:, 2])
plt.plot(phi_0/np.pi, eigv_c[:, 3])

In [ ]:
# current matrix for continuum states E > Delta - retarded square root in the denominator

def curr_matr_cont(x, i):
    
    s_L = 1
    s_R = -1
    zero_m = np.zeros_like(Gamma_L)
    
    Lambda_L = x * np.block([[Gamma_L , zero_m], [zero_m, Gamma_L]]) \
             - Delta * np.block([[zero_m, np.exp(-1j * s_L * phi_0[i]/2) * Gamma_L], \
                                [np.exp(1j * s_L * phi_0[i]/2) * Gamma_L, zero_m]])
    
    Lambda_R = x * np.block([[Gamma_R , zero_m], [zero_m, Gamma_R]]) \
             - Delta * np.block([[zero_m, np.exp(-1j * s_R * phi_0[i]/2) * Gamma_R], \
                                [np.exp(1j * s_R * phi_0[i]/2) * Gamma_R, zero_m]])
    
    I = -1 / (-1j * np.sign(x) * np.sqrt(x**2 - Delta**2)) / (2*1j) \
        * (s_L * Lambda_L + s_R * Lambda_R)
    
    return I

In [ ]:
# continuum current matrix element

def curr_cont_nl(E, E_lamd, psi_n, eta_lamd, i):
    
    tau_z = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, -1, 0], [0, 0, 0, -1]])
    
    matr = tau_z @ curr_matr(E_lamd, i) - curr_matr_cont(E, i) @ tau_z
    
    return np.conj(psi_n) @ matr @ eta_lamd    

In [ ]:
def integr_gamma_out(x, E_lamd, eta_lamd, i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c):
    
    E = x + E_lamd
    
    # 4 eigenvalues, eigenvectors for continuum state
    eigv_E, eigvect_E = spectr(E, i)
    
    # current elements
    I_nl = np.zeros((4), dtype='complex')
    
    for n in range(4): 
        I_nl = curr_cont_nl(E, E_lamd, eigvect_E[:, n], eta_lamd, i)
        
    sum_n = np.sum(eigv_E * np.abs(I_nl)**2)
    
    n_F = fermi_distrib(E, T_ferm)
    n_B = bose_distrib(x, T_bos)
    J_sp = spectral_density(x, lambda_coupl, alpha_0, damping_param, Omega, omega_c)
    
    return J_sp * n_B * (1 - n_F) * sum_n 
    

In [ ]:
def integr_gamma_in(x, E_lamd, eta_lamd, i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c):
    
    E = x + E_lamd
    
    # 4 eigenvalues, eigenvectors for continuum state
    eigv_E, eigvect_E = spectr(E, i)
    
    # current elements
    I_nl = np.zeros((4), dtype='complex')
    
    for n in range(4): 
        I_nl = curr_cont_nl(E, E_lamd, eigvect_E[:, n], eta_lamd, i)
        
    sum_n = np.sum(eigv_E * np.abs(I_nl)**2)
    
    n_F = fermi_distrib(E, T_ferm)
    n_B = bose_distrib(x, T_bos)
    J_sp = spectral_density(x, lambda_coupl, alpha_0, damping_param, Omega, omega_c)
    
    return J_sp * (n_B + 1) * n_F * sum_n 
    

In [ ]:
def integr_gamma(x, E_lamd, eta_lamd, i):
    
    E = x + E_lamd
    
    # 4 eigenvalues, eigenvectors for continuum state
    eigv_E, eigvect_E = spectr(E, i)
    
    # current elements
    I_nl = np.zeros((4), dtype='complex')
    
    for n in range(4): 
        I_nl = curr_cont_nl(E, E_lamd, eigvect_E[:, n], eta_lamd, i)
        
    sum_n = np.sum(eigv_E * np.abs(I_nl)**2)
    
    n_F = fermi_distrib(E, T_ferm)
    n_B = bose_distrib(x, T_bos)
    J_sp = spectral_density(x, lambda_coupl, alpha_0, damping_param, Omega, omega_c)
    
    G_out = J_sp * n_B * (1 - n_F) * sum_n
    G_in = J_sp * (n_B + 1) * n_F * sum_n
    
    return np.array([G_out, G_in])

    

Calculate $\Gamma_{\lambda}^{\text{out/in}}$ for each Andreev level $\lambda \in 2l$:

In [ ]:
Gamma_out_pc = np.zeros((len(phi_0), 2))
Gamma_out_mc = np.zeros_like(Gamma_out_pc) 
Gamma_in_pc = np.zeros_like(Gamma_out_pc)
Gamma_in_mc = np.zeros_like(Gamma_out_pc)

In [ ]:
def compute_integrals(i, lamd, Delta, roots, eta_arr, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c):
    
    Gamma_out_pc = integrate.quad(integr_gamma_out, Delta - roots[i, lamd], np.infty,
                                  args=(roots[i, lamd], eta_arr[lamd, i], i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c))[0]
    Gamma_out_mc = integrate.quad(integr_gamma_out, -np.infty, -Delta - roots[i, lamd],
                                  args=(roots[i, lamd], eta_arr[lamd, i], i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c))[0]
    Gamma_in_pc = integrate.quad(integr_gamma_in, Delta - roots[i, lamd], np.infty,
                                 args=(roots[i, lamd], eta_arr[lamd, i], i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c))[0]
    Gamma_in_mc = integrate.quad(integr_gamma_in, -np.infty, -Delta - roots[i, lamd],
                                 args=(roots[i, lamd], eta_arr[lamd, i], i, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c))[0]
    
    # Return results as a tuple
    return (i, lamd, Gamma_out_pc, Gamma_out_mc, Gamma_in_pc, Gamma_in_mc)

In [ ]:
# Use ProcessPoolExecutor for parallel execution
with ProcessPoolExecutor() as executor:
    
    # Create a list of futures by submitting tasks for parallel execution
    futures = [executor.submit(compute_integrals, i, lamd, Delta, roots, eta_arr, T_bos, T_ferm,\
                    lambda_coupl, alpha_0, damping_param, Omega, omega_c)
               for i in range(len(phi_0)) for lamd in reversed(range(2))]
    
    # Collect results as they are completed
    for future in futures:
        i, lamd, out_pc, out_mc, in_pc, in_mc = future.result()
        
        # Store results in the appropriate arrays
        Gamma_out_pc[i, lamd] = out_pc
        Gamma_out_mc[i, lamd] = out_mc
        Gamma_in_pc[i, lamd] = in_pc
        Gamma_in_mc[i, lamd] = in_mc

In [ ]:
Gamma_in = (Gamma_in_pc + Gamma_in_mc) * 2*np.pi
Gamma_out = (Gamma_out_pc + Gamma_out_mc) * 2*np.pi

Gamma_in = np.tile(Gamma_in, 2)
Gamma_out = np.tile(Gamma_out, 2)

In [ ]:
plt.plot(phi_0/np.pi, Gamma_out[:, 0], label=r'$\Gamma^{out}_1$')
plt.plot(phi_0/np.pi, Gamma_in[:, 0], label=r'$\Gamma^{in}_1$')

plt.plot(phi_0/np.pi, Gamma_out[:, 1], label=r'$\Gamma^{out}_{2}$')
plt.plot(phi_0/np.pi, Gamma_in[:, 1], label=r'$\Gamma^{in}_{2}$')

plt.legend()

In [ ]:
plt.scatter(roots[:, 0], Gamma_out[:, 0], label=r'$\Gamma^{out}_0$', s=8)
plt.scatter(roots[:, 0], Gamma_in[:, 0], label=r'$\Gamma^{in}_0$', s=8)
plt.scatter(roots[:, 1], Gamma_out[:, 1], label=r'$\Gamma^{out}_1$', s=8)
plt.scatter(roots[:, 1], Gamma_in[:, 1], label=r'$\Gamma^{in}_1$', s=8)

plt.legend()
plt.grid()
plt.yscale("log")

## Lindblad euqation: steady state

In [ ]:
N = 4
N_state = 2**N

In [ ]:
# function that returns the momentum state (as string) for printing

def print_state(n):
    
    if len(bin(n)[:1:-1]) < N:
        new_bin = bin(n)[:1:-1]
        for i in range(N - len(bin(n)[:1:-1])):
            new_bin += '0'
            
        return new_bin
    else:
        return bin(n)[:1:-1]

In [ ]:
# the momentum vectors < q_i0 ... q_iNk | rho | q_j0 ... q_jNk >, where q's can be either 0 or 1 
# in case of fermions - by writing each i and j of the density matrix in the binary representation
# we get all possible combinations of occupied/non-occupied Nk momentum states

# this function finds whether the q'_iq entry (q = [0, Nk-1] index of the entry in the vector) is occupied or not 
def n(q, i):
    if q > len(bin(i)[:1:-1])-1:
        return 0
    else:
        return int(bin(i)[:1:-1][q])

In [ ]:
lvl = np.array(range(N))
states = np.array(range(N_state))

n_v = np.vectorize(n)

nf_h = np.array([n_v(lvl, si) for si in states])

In [ ]:
# steady state
Steady_M = np.zeros((len(phi_0), N_state, N_state), dtype='float64') 

for j in range(len(phi_0)):
    
    # loop over the raws - as many as there are states - 16
    for i in range(N_state):

        # loop over 4 levels
        for mu in range(4):

            n_mu = nf_h[i, mu]

            # G_in
            Steady_M[j, i, i + (-1)**n_mu * 2**mu] += n_mu * (Gamma_in[j, mu] )

            # G_out
            Steady_M[j, i, i + (-1)**n_mu * 2**mu] += (1-n_mu) * (Gamma_out[j, mu] )

            # G_in, G_out
            Steady_M[j, i, i] -= n_mu * (Gamma_out[j, mu]) + \
                                (1-n_mu) * (Gamma_in[j, mu] )

    #             loop over lambda' != lamda
            for nu in chain(range(mu), range(mu+1, 4)):

                n_nu = nf_h[i, nu]

                Steady_M[j, i, i + (-1)**n_mu * 2**mu + (-1)**n_nu * 2**nu] += \
                                n_mu * (1-n_nu) * G_AL_pp[j, nu, mu] + \
                                n_mu * n_nu * G_AL_mp[j, nu, mu] + \
                                (1-n_mu) * (1-n_nu) * G_AL_pm[j, nu, mu]

                Steady_M[j, i, i] -= (1-n_mu) * n_nu * G_AL_pp[j, nu, mu] + \
                                         (1-n_mu) * (1-n_nu) * G_AL_mp[j, nu, mu] +\
                                         n_mu * n_nu * G_AL_pm[j, nu, mu]


In [ ]:
def eig_M(M):
    
    eigval_M = np.zeros((16), dtype='complex')
    eigvec_M = np.zeros((16, 16), dtype='complex')
    
    eigval_M, eigvec_M = LA.eig(M)
        
    return eigval_M, eigvec_M

In [ ]:
def find_c(init, eigvec_M):
    
    coeff = np.zeros((16))
    
    Matr = np.zeros((16, 16))
        
    for j in range(16):
        Matr[j] = eigvec_M[:, j]
            
    coeff = init @ LA.inv(Matr)
        
    return coeff

In [ ]:
def time_dep(t, init, eigval_M, eigvec_M):
    P_t = np.zeros((16))
    coeff = find_c(init, eigvec_M)
    
    for j in range(16):
        P_t[j] = np.sum(np.exp(eigval_M * t) * coeff @ eigvec_M[j, :])
        
        
    return P_t       

In [ ]:
eigval_M, eigvec_M = eig_M(Steady_M)

In [ ]:
for j in range(16):
    plt.scatter(phi_0/np.pi, eigval_M[:, j], s=5)

In [ ]:
ns = np.zeros((len(phi_0), 16), dtype='float64')

for i in range(len(phi_0)):
    ns[i] = null_space(Steady_M[i], rcond=10e-16)[:,0]
    
steady_occup = ns
steady_occup /= np.sum(steady_occup, axis=1, keepdims=True)

In [ ]:
# note here the numbering of states:

#   1  ----------    ----------  3  
#   0  ----------    ----------  2 

# ----------------------------------

for i in range(N_state):
    print(nf_h[i], '\t', steady_occup[51, i])

In [ ]:
plt.plot(phi_0/np.pi, steady_occup[:, 0], label=r'$P_0$')
plt.plot(phi_0/np.pi, steady_occup[:, 1], label=r'$P_1$')
plt.plot(phi_0/np.pi, steady_occup[:, 2], label=r'$P_2$')
plt.plot(phi_0/np.pi, steady_occup[:, 3], label=r'$P_3$')
plt.plot(phi_0/np.pi, steady_occup[:, 4], label=r'$P_4$')
plt.plot(phi_0/np.pi, steady_occup[:, 5], label=r'$P_5$')
plt.plot(phi_0/np.pi, steady_occup[:, 6], label=r'$P_6$')

plt.grid()
plt.legend()

### Compue the relaxation times

In [ ]:
def rhs1(t, P, i_eq):

    # find derivatives of N_state elements
    dP = np.zeros(N_state)
        
    dP = Steady_M[i_eq] @ P
        
    return dP

In [ ]:
def compute_tau(i1, i2):
    
    P0 = steady_occup[i1]
    sol_ = solve_ivp(rhs1, (0.0, 1e13), P0, method='LSODA', dense_output=True, args=(i2,))
    ts_ = sol_.t
    ys_ = np.stack(sol_.y)
    
    tau = 1e10  # Default value for tau
    for j in range(len(ts_)):
        if np.sum(np.abs(ys_[:, j] - steady_occup[i2, :])) < 1e-3:
            tau = ts_[j]
            break
            
    return i1, i2, tau

In [ ]:
tau_arr = np.zeros((len(phi_0), len(phi_0)))

# Parallel execution
with ProcessPoolExecutor() as executor:
    # Submit tasks for all (i1, i2) pairs
    futures = [executor.submit(compute_tau, i1, i2) for i1 in range(len(phi_0)) for i2 in range(len(phi_0))]
    
    # Collect results and populate tau_arr
    for future in futures:
        i1, i2, tau = future.result()
        tau_arr[i1, i2] = tau

# Now tau_arr contains the computed tau values

In [ ]:
plt.imshow(tau_arr, extent=[phi_0.min()/np.pi, phi_0.max()/np.pi, phi_0.min()/np.pi, phi_0.max()/np.pi],\
           origin='lower', aspect='auto', cmap='turbo',)

plt.colorbar(label=r'$\tau, \Delta$')
plt.xlabel(r'$\varphi_{init}, \pi$')
plt.ylabel(r'$\varphi_{final}, \pi$')

plt.title(r"$L = {}\Delta^{{-1}}, t_L = {}, t_R = {}, $"\
          .format(L, t_L, t_R)+'\n'+
          r"$\Omega={}\Delta, T_{{qp}} = {}\Delta, T_b = {}\Delta$"\
          .format(Omega, T_ferm, T_bos), fontsize=13)
# plt.title('Color 2D Plot of Tau Array')
plt.show()

In [ ]:
tau_arr_log = np.log10(np.maximum(tau_arr, 10))

In [ ]:
plt.imshow(tau_arr_log, extent=[phi_0.min()/np.pi, phi_0.max()/np.pi, phi_0.min()/np.pi, phi_0.max()/np.pi],
           origin='lower', aspect='auto', cmap='viridis', vmin=1)
plt.title(r"$L = {}\Delta^{{-1}}, t_L = {}, t_R = {}, $"\
          .format(L, t_L, t_R)+'\n'+
          r"$\Omega={}\Delta, T_{{qp}} = {}\Delta, T_b = {}\Delta$"\
          .format(Omega, T_ferm, T_bos), fontsize=13)

cbar = plt.colorbar()
cbar.set_label(r'$\log_{10}(\tau)$')  # Color bar label in log scale

plt.xlabel(r'$\varphi_{init}$')
plt.ylabel(r'$\varphi_{final}$')

plt.show()

In [ ]:
P0 = steady_occup[:, np.newaxis]
init_trace_dist = np.sum(np.abs(P0 - steady_occup), axis=2)

In [ ]:
plt.imshow(init_trace_dist, extent=[phi_0.min()/np.pi, phi_0.max()/np.pi, phi_0.min()/np.pi, phi_0.max()/np.pi],
           origin='lower', aspect='auto', cmap='jet', vmin=0)
plt.title(r"$L = {}\Delta^{{-1}}, t_L = {}, t_R = {}, $"\
          .format(L, t_L, t_R)+'\n'+
          r"$\Omega={}\Delta, T_{{qp}} = {}\Delta, T_b = {}\Delta$"\
          .format(Omega, T_ferm, T_bos), fontsize=13)

cbar = plt.colorbar()
cbar.set_label(r'$\mathcal{D}_T(0)$')  # Color bar label in log scale

plt.xlabel(r'$\varphi_{init}, \pi$')
plt.ylabel(r'$\varphi_{final}, \pi$')

plt.show()

### Compute the time evolution of the trace distance

In [ ]:
# Helper function to compute trace distance for each (i1, i2) pair

def compute_trace_distance(i1, i2, phi_0, steady_occup):
    
    P0 = steady_occup[i1]
    sol_ = solve_ivp(rhs1, (0.0, 1.5e5), P0, method='LSODA', dense_output=True, args=(i2,))
    
    ts_ = sol_.t
    ys_ = np.stack(sol_.y)
    
    steady_reshaped = np.tile(steady_occup[i2, :, np.newaxis], (1, len(ts_)))
    trace_distance = np.sum(np.abs(ys_ - steady_reshaped), axis=0)
    
    return i1, i2, trace_distance, ts_

In [ ]:
# Initialize trace_dist with the correct shape (assuming it's a 2D array)

trace_dist = np.zeros((len(phi_0), len(phi_0)), dtype=object)
time = np.zeros((len(phi_0), len(phi_0)), dtype=object)

In [ ]:
# Use ProcessPoolExecutor to parallelize the computation

with ProcessPoolExecutor() as executor:
    futures = [
        executor.submit(compute_trace_distance, i1, i2, phi_0, steady_occup)
        for i1 in range(len(phi_0))
        for i2 in range(len(phi_0))
    ]
    for future in futures:
        i1, i2, trace_distance, ts_ = future.result()
        trace_dist[i1][i2] = trace_distance
        time[i1][i2] = ts_        

In [ ]:
plt.plot(time[20][38], trace_dist[20][38])
plt.plot(time[20][26], trace_dist[20][26])
plt.plot(time[20][24], trace_dist[20][24])

plt.yscale('log')


plt.grid()

In [ ]:
time_test = np.linspace(1, 1.5e5, num=5000)
P_t = np.zeros((len(time_test), 16))
P_t1 = np.zeros((len(time_test), 16))
P_t2 = np.zeros_like(P_t)

In [ ]:
for t in range(len(time_test)):
    P_t[t] = time_dep(time_test[t], steady_occup[20], eigval_M[26], eigvec_M[26])
    P_t1[t] = time_dep(time_test[t], steady_occup[20], eigval_M[38], eigvec_M[38])
    P_t2[t] = time_dep(time_test[t], steady_occup[20], eigval_M[24], eigvec_M[24])    

In [ ]:
plt.plot(time_test, np.sum(np.abs(P_t2-steady_occup[24]), axis=1), label=r'$\varphi_i = ${}, $\varphi_f = ${}'.\
         format(phi_0[20]/np.pi, np.round(phi_0[24]/np.pi, 2)))
plt.plot(time_test, np.sum(np.abs(P_t-steady_occup[26]), axis=1), label=r'$\varphi_i = ${}, $\varphi_f = ${}'.\
         format(phi_0[20]/np.pi, np.round(phi_0[26]/np.pi, 2)))
plt.plot(time_test, np.sum(np.abs(P_t1-steady_occup[38]), axis=1), label=r'$\varphi_i = ${}, $\varphi_f = ${}'.\
         format(phi_0[20]/np.pi, np.round(phi_0[38]/np.pi, 2)))




plt.title(r"$L = {}\Delta^{{-1}}, t_L = {}, t_R = {}, $"\
          .format(L, t_L, t_R)+'\n'+
          r"$\Omega={}\Delta, T_{{qp}} = {}\Delta, T_b = {}\Delta$"\
          .format(Omega, T_ferm, T_bos), fontsize=13)


plt.ylabel(r'$\mathcal{D}_T(t)$')
plt.legend()
plt.yscale('log')

# plt.xlim(-1000, 3e4)
plt.grid()